In [8]:
eventsPath = os.environ["YAHOO_DATA"]
execfile("../script/utils.py")
splitedRddMerged = sc.textFile(eventsPath + "splitedDataMerged")
splitedRddMergedParsed = splitedRddMerged.map(parseContextData2Merged)
import json
itemIdConversionDictionaryMerged = json.load(open(eventsPath + "itemIdConversionDictionaryMerged.txt"))
#def filter_func(line,factor):
#    data = line[1]
#    if len(data[1]) == 0:
#        return False
#    return len(data[2])/float(len(data[1])) > factor
#splitedRddMergedParsed10p = splitedRddMergedParsed.filter(lambda x: filter_func(x,0.1))
#splitedRddMergedParsed10p.count()

In [10]:
def parseTopGlobalApps(line):
    els = line.split(',')
    return els[0]
topglobalapps = sc.textFile(eventsPath + "stats/topapps_run_byusers.csv").map(parseTopGlobalApps).take(11)[1:]
topglobalappsIds = [itemIdConversionDictionaryMerged[x] for x in topglobalapps]
topglobalapps

[u'com.android.mms',
 u'com.facebook.katana',
 u'com.android.contacts',
 u'com.android.phone',
 u'com.android.systemui',
 u'com.android.chrome',
 u'com.android.incallui',
 u'com.android.settings',
 u'com.instagram.android',
 u'com.google.android.googlequicksearchbox']

In [13]:
splitratio = 0.8
def filterTopGlobalApps((user,events)):
    eventTemp = events[0] + events[1]
    eventTemp = [x for x in eventTemp if x[0] not in topglobalappsIds]
    l = int(len(eventTemp) * splitratio)
    return (user, [eventTemp[:l], eventTemp[l:], events[2], events[3]])
splitedRddMergedParsedFiltered = splitedRddMergedParsed.map(filterTopGlobalApps)


In [15]:
splitedRddMergedParsedFiltered.count()

196335

In [17]:
splitedRddSample1 = sc.parallelize(splitedRddMergedParsedFiltered.takeSample(True, 2000,0))
splitedRddSample2 = sc.parallelize(splitedRddMergedParsedFiltered.takeSample(True, 2000,1))

In [18]:
numRecList = 5
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output
def topFiveSortedList(oldList, context, probability):
    context_oldList = [x[0] for x in oldList]
    if context not in context_oldList:
        for x in range(0,len(oldList)-1):
            if x == 0 and oldList[x][1] < probability:
                oldList[x] = (context,probability)
                if oldList[x+1][1] < oldList[x][1]:
                    temp = oldList[x+1] 
                    oldList[x+1] = oldList[x]
                    oldList[x] = temp 
                else:
                    break
            elif oldList[x+1][1] < oldList[x][1]:
                temp = oldList[x+1] 
                oldList[x+1] = oldList[x]
                oldList[x] = temp 
            else:
                break
    else:
        index = context_oldList.index(context)
        if oldList[index][1] < probability:
            oldList[index] = (context,probability)
        oldList = sorted(oldList,key=lambda x: x[1])
    return oldList     
def evaluation_rr(testList, recList):
    scores = 0
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                scores = scores + 1.0 / (i+1)
                break
    if len(testList) != 0 :
        scores = scores / len(testList)
    return scores
def evaluation_rr1(testItem, recList):
    scores = 0
    for i in range(len(recList)):
        if testItem == recList[i]:
            scores = 1.0 / (i+1)
            break
    return scores
def evaluation_ap(testList, recList):
    scores = 0
    num = 0 
    rr = []
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    if rr[j] == 1:
                        scores = scores + 1.0 / (j+1)
                        num = num + 1
                break
    if num == 0:
        scores = 0
    else:
        scores = scores / num
    return scores

def evaluation_rap(testList,recList):
    scores = 0.0
    hitNum = 0  
    for i in range(len(recList)):
        if recList[i] in testList:
            hitNum = hitNum + 1
            scores = scores + 1.0*hitNum/(i+1)
    if hitNum != 0:
        scores = scores/hitNum
    else:
        scores = 0
    return scores

def evaluation_recall(testList, recList):
    scores = 0
    num = 0
    recall = []
    recall[:len(recList)] = [0 for i in range(len(recList))]
    rr=[]
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    recall[j] = recall[j] + 1.0
                    num = num + 1  
                break    
    recall = [0 if len(testList) == 0 else x/len(testList) for x in recall]
    return recall
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output

def evaluation_precision(testItem, recList):
    scores = 0
    if testItem in recList:
        scores = 1.0
    return scores  


def print_result(finalScore, algorithm_name):
    f = open('result/mfu_test_result6.txt','a')
    f.write('10P type of users with only evaluation after installed')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by mrr1:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[2]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[3][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()))
            
    f.write(algorithm_name + ' by precision:'+str(finalScore.map(lambda x:x[4]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by rap:'+str(finalScore.map(lambda x:x[5]).mean()))
    f.write('\n')
    f.close()

In [23]:
from operator import add
from collections import Counter
f = open('result/mfu_test_result6.txt','a')
f.write('-------normal mfu with global top 10 apps removed-----')
f.write('\n')
f.close()
def mfuFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    
    scores_precision = 0.0
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    
    trainList = listGroup[0]     #0.7 train set
    testList = listGroup[1]    #0.3 test set
    numTrain = len(trainList)
    numTest = len(testList)
    
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender = Counter(RecommenderDuplicate).most_common()
        Recommender =  [t[0] for t in Recommender] 

        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
            
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        trainList = sorted(trainList,key=lambda x:int(x[1][0])); ####
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_rr1 = scores_rr1 / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap

finalScore = splitedRddSample1.map(mfuFunction)
print_result(finalScore,'mfu_sample1')
finalScore = splitedRddSample2.map(mfuFunction)
print_result(finalScore,'mfu_sample2')

In [22]:
f = open('result/mfu_test_result6.txt','a')
f.write('-------bayesian_moving with global top 10 apps removed----')
f.write('\n')
f.close()
def bayesian_moving(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][4]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][4]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext # /numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi   #/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set
        
        if len(testList_items)>5:
            testList_items = testList_items[:5]
        
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_rr1 = scores_rr1 / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr,scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
#    return line[2][1]
finalScore = splitedRddSample1.map(bayesian_moving)
print_result(finalScore, 'bayesian_moving_sample1')
finalScore = splitedRddSample2.map(bayesian_moving)
print_result(finalScore, 'bayesian_moving_sample2')

KeyboardInterrupt: 

In [24]:
f = open('result/mfu_test_result6.txt','a')
f.write('-------bayesian_moving with global top 10 apps removed----')
f.write('\n')
f.close()
def bayesian_location(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][5]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][5]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext # /numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi   #/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set
        
        if len(testList_items)>5:
            testList_items = testList_items[:5]
        
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_rr1 = scores_rr1 / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr,scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
#    return line[2][1]
finalScore = splitedRddSample1.map(bayesian_location)
print_result(finalScore, 'bayesian_location_sample1')
finalScore = splitedRddSample2.map(bayesian_moving)
print_result(finalScore, 'bayesian_location_sample2')

In [75]:
def evaluation_ap(testList, recList):
    scores = 0
    num = 0 
    rr = []
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    if rr[j] == 1:
                        scores = scores + 1.0 / (j+1)
                        num = num + 1
                break
    if num == 0:
        scores = 0
    else:
        scores = scores / num
    return scores
a = [1,2,3,11,12]
b = [1,2,3,4,5]
evaluation_ap(a,b)

0.5

In [74]:
def evaluation_rap(testList,recList):
    scores = 0.0
    hitNum = 0  
    for i in range(len(recList)):
        if recList[i] in testList:
            hitNum = hitNum + 1
            scores = scores + 1.0*hitNum/(i+1)
    if hitNum != 0:
        scores = scores/hitNum
    else:
        scores = 0
    return scores

a = [1,2,3,11,12]
b = [1,2,3,4,5]
evaluation_rap(a,b)

1.0